# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head(10)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
5,5,Bryan Miranda,M,9th,Huang High School,94,94,0,District,2917,1910635
6,6,Sheena Carter,F,11th,Huang High School,82,80,0,District,2917,1910635
7,7,Nicole Baker,F,12th,Huang High School,96,69,0,District,2917,1910635
8,8,Michael Roth,M,10th,Huang High School,95,87,0,District,2917,1910635
9,9,Matthew Greene,M,10th,Huang High School,96,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
#Pandas Unit 3 - Bugfixing Bonanza
total_schools = school_data["school_name"].count()
total_students = school_data_complete["student_name"].count()
total_budget = school_data["budget"].sum()
avg_math_score = school_data_complete["math_score"].mean()
avg_reading_score = school_data_complete["reading_score"].mean()

# Passing math score 70 +

passing_math = school_data_complete.loc[(student_data["math_score"] >= 70)]
count_passing_math = passing_math["math_score"].count()
percent_math_pass = (count_passing_math/total_students)*100

# Passing reading score 70 +

passing_reading = school_data_complete.loc[(student_data["reading_score"] >= 70)]
count_passing_reading = passing_reading["reading_score"].count()
percent_reading_pass = (count_passing_reading/total_students)*100

#Overall passing rate (0verall score)

overall_passing_rate = (count_passing_math + count_passing_reading)/2

# Create new data fram to mirror the table in the homework instructions

district_summary = {"Total Schools" : total_schools,
                   "Total Students" : total_students,
                   "Total Budget" : total_budget,
                   "Average Math Score" : avg_math_score,
                   "Average Reading Score" : avg_reading_score,
                   "% Passing Math" : percent_math_pass,
                   "% Passing Reading" : percent_reading_pass,
                   "% Overall Passing Rate" : overall_passing_rate}
district_summary_df = pd.DataFrame([district_summary])
district_summary_df = district_summary_df[["Total Schools", "Total Students", "Total Budget", "Average Math Score",
                                          "Average Reading Score", "% Passing Math", "% Passing Reading",
                                          "% Overall Passing Rate"]]
district_summary_df


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,31490.0


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [3]:
#School Summary

copy_school_sum = school_data.copy()
avg_math_reading_table = school_data_complete.groupby(['school_name'])['reading_score', 
                                                                       'math_score'].mean().reset_index()
copy_school_sum['Per Student Budget'] = copy_school_sum['budget']/copy_school_sum['size']

copy_school_sum = copy_school_sum.merge(avg_math_reading_table, on='school_name', how="outer")

summary_passing_reading = school_data_complete[school_data_complete['reading_score'] >=70]

summary_passing_math = school_data_complete[school_data_complete['math_score'] >=70]

# How many students are passing in reading & math?

count_pass_reading = summary_passing_reading.groupby(["school_name"])['reading_score'].count().reset_index()

count_pass_math = summary_passing_math.groupby(["school_name"])['math_score'].count().reset_index()


# Rename columns to Math and Reading Count

count_pass_reading.rename_axis({'reading_score' : 'Reading Count'}, axis=1, inplace=True)

count_pass_math.rename_axis({'math_score' : 'Math Count'}, axis=1, inplace=True)


# Overall Pass Rate -- Need to merge

overall_pass_count = count_pass_math.merge(count_pass_reading, on="school_name", how='outer')

copy_school_sum = copy_school_sum.merge(overall_pass_count, on="school_name", how='outer')


# % Passing Reading and Math

copy_school_sum['% Passing Reading'] = (copy_school_sum['Reading Count']/copy_school_sum['size'])*100
copy_school_sum['% Passing Math'] = (copy_school_sum['Math Count']/copy_school_sum['size'])*100


# % Overall Pass Rate

copy_school_sum['% Overall Passing Rate'] = (copy_school_sum['% Passing Math'] + copy_school_sum['% Passing Reading'])/2

# Needed to name columns

copy_school_sum.rename_axis({'reading_score' : 'Average Reading Score', 'math_score' : 'Average Math Score'}, axis=1, inplace=True)
copy_school_sum.rename_axis({'school_name' : 'School Name'}, axis=1, inplace=True)
copy_school_sum.rename_axis({'type' : 'School Type'}, axis=1, inplace=True)
copy_school_sum.rename_axis({'size' : 'Size'}, axis=1, inplace=True)
copy_school_sum.rename_axis({'budget' : 'School Budget'}, axis=1, inplace=True)

# I noticed that I needed to delete some columns from the table

del copy_school_sum['Math Count']
del copy_school_sum['Reading Count']
del copy_school_sum['School ID']


copy_school_sum.head(10)

C:\Users\User\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: Using 'rename_axis' to alter labels is deprecated. Use '.rename' instead
C:\Users\User\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: Using 'rename_axis' to alter labels is deprecated. Use '.rename' instead
C:\Users\User\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:47: FutureWarning: Using 'rename_axis' to alter labels is deprecated. Use '.rename' instead
C:\Users\User\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:48: FutureWarning: Using 'rename_axis' to alter labels is deprecated. Use '.rename' instead
C:\Users\User\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:49: FutureWarning: Using 'rename_axis' to alter labels is deprecated. Use '.rename' instead
C:\Users\User\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:50: FutureWarning: Using 'rename_axis' to alter labels is de

,School Name,School Type,Size,School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Overall Passing Rate
0,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,81.316421,65.683922,73.500171
1,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,80.739234,65.988471,73.363852
2,Shelton High School,Charter,1761,1056600,600.0,83.725724,83.359455,95.854628,93.867121,94.860875
3,Hernandez High School,District,4635,3022020,652.0,80.934412,77.289752,80.862999,66.752967,73.807983
4,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,97.138965,93.392371,95.265668
5,Wilson High School,Charter,2283,1319574,578.0,83.989488,83.274201,96.539641,93.867718,95.203679
6,Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,97.039828,94.133477,95.586652
7,Bailey High School,District,4976,3124928,628.0,81.033963,77.048432,81.933280,66.680064,74.306672
8,Holden High School,Charter,427,248087,581.0,83.814988,83.803279,96.252927,92.505855,94.379391
9,Pena High School,Charter,962,585858,609.0,84.044699,83.839917,95.945946,94.594595,95.270270


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [4]:
# Top Performing Schools (by passing rate)

top_performing_pass_rate = copy_school_sum.sort_values(by=['% Overall Passing Rate'], ascending=False).head(10)
top_performing_pass_rate

,School Name,School Type,Size,School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Overall Passing Rate
6,Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,97.039828,94.133477,95.586652
14,Thomas High School,Charter,1635,1043130,638.0,83.848930,83.418349,97.308869,93.272171,95.290520
9,Pena High School,Charter,962,585858,609.0,84.044699,83.839917,95.945946,94.594595,95.270270
4,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,97.138965,93.392371,95.265668
5,Wilson High School,Charter,2283,1319574,578.0,83.989488,83.274201,96.539641,93.867718,95.203679
10,Wright High School,Charter,1800,1049400,583.0,83.955000,83.682222,96.611111,93.333333,94.972222
2,Shelton High School,Charter,1761,1056600,600.0,83.725724,83.359455,95.854628,93.867121,94.860875
8,Holden High School,Charter,427,248087,581.0,83.814988,83.803279,96.252927,92.505855,94.379391
7,Bailey High School,District,4976,3124928,628.0,81.033963,77.048432,81.933280,66.680064,74.306672
3,Hernandez High School,District,4635,3022020,652.0,80.934412,77.289752,80.862999,66.752967,73.807983


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [5]:
worst_performing_pass_rate = copy_school_sum.sort_values(by=['% Overall Passing Rate']).head(10)
worst_performing_pass_rate

,School Name,School Type,Size,School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Overall Passing Rate
11,Rodriguez High School,District,3999,2547363,637.0,80.744686,76.842711,80.220055,66.366592,73.293323
1,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,80.739234,65.988471,73.363852
0,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,81.316421,65.683922,73.500171
12,Johnson High School,District,4761,3094650,650.0,80.966394,77.072464,81.222432,66.057551,73.639992
13,Ford High School,District,2739,1763916,644.0,80.746258,77.102592,79.299014,68.309602,73.804308
3,Hernandez High School,District,4635,3022020,652.0,80.934412,77.289752,80.862999,66.752967,73.807983
7,Bailey High School,District,4976,3124928,628.0,81.033963,77.048432,81.933280,66.680064,74.306672
8,Holden High School,Charter,427,248087,581.0,83.814988,83.803279,96.252927,92.505855,94.379391
2,Shelton High School,Charter,1761,1056600,600.0,83.725724,83.359455,95.854628,93.867121,94.860875
10,Wright High School,Charter,1800,1049400,583.0,83.955000,83.682222,96.611111,93.333333,94.972222


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [33]:
# Math scores by grade

#math_nineth = student_data.loc[student_data['grade'] == '9th'].groupby('school_name')['math_score'].mean()
#math_tenth = student_data.loc[student_data['grade'] == '10th'].groupby('school_name')['math_score'].mean()
#math_eleventh = student_data.loc[student_data['grade'] == '11th'].groupby('school_name')['math_score'].mean()
#math_twelfth = student_data.loc[student_data['grade'] == '12th'].groupby('school_name')['math_score'].mean()

#math_scores_df = pd.DataFrame({"9th" : math_nineth, 
#                              "10th" : math_tenth,
#                              "11th" : math_eleventh,
#                              "12th" : math_twelfth})
#math_scores_df = ['9th', '10th', '11th', '12th']


#math_scores_df.head(10)

math_nineth = student_data.loc[student_data['grade'] == '9th'].groupby('school_name')['math_score'].mean()
math_tenth = student_data.loc[student_data['grade'] == '10th'].groupby('school_name')['math_score'].mean()
math_eleventh = student_data.loc[student_data['grade'] == '11th'].groupby('school_name')['math_score'].mean()
math_twelfth = student_data.loc[student_data['grade'] == '12th'].groupby('school_name')['math_score'].mean()

math_scores_df = pd.DataFrame({"9th" : math_nineth, 
                               "10th" : math_tenth,
                               "11th" : math_eleventh,
                               "12th" : math_twelfth})

math_scores_grade = school_data_complete.groupby(["school_name"])['grade'].count().reset_index()
math_scores_grade


,school_name,grade
0,Bailey High School,4976
1,Cabrera High School,1858
2,Figueroa High School,2949
3,Ford High School,2739
4,Griffin High School,1468
5,Hernandez High School,4635
6,Holden High School,427
7,Huang High School,2917
8,Johnson High School,4761
9,Pena High School,962


* Perform the same operations as above for reading scores

In [34]:
reading_nineth = student_data.loc[student_data['grade'] == '9th'].groupby('school_name')['reading_score'].mean()
reading_tenth = student_data.loc[student_data['grade'] == '10th'].groupby('school_name')['reading_score'].mean()
reading_eleventh = student_data.loc[student_data['grade'] == '11th'].groupby('school_name')['reading_score'].mean()
reading_twelfth = student_data.loc[student_data['grade'] == '12th'].groupby('school_name')['reading_score'].mean()

reading_scores_df = pd.DataFrame({"9th" : reading_nineth, 
                               "10th" : reading_tenth,
                               "11th" : reading_eleventh,
                               "12th" : reading_twelfth})

reading_scores_grade = school_data_complete.groupby(["school_name"])['grade'].count().reset_index()
reading_scores_grade

,school_name,grade
0,Bailey High School,4976
1,Cabrera High School,1858
2,Figueroa High School,2949
3,Ford High School,2739
4,Griffin High School,1468
5,Hernandez High School,4635
6,Holden High School,427
7,Huang High School,2917
8,Johnson High School,4761
9,Pena High School,962


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [38]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [47]:
#print(school_data_complete["math_score"].mean())
#print(school_data_complete["reading_score"].mean())

school_spending_bins = pd.cut(copy_school_sum['Per Student Budget'], spending_bins, labels=group_names)

school_spending_bins = pd.DataFrame(school_spending_bins)

copy_school_sum['Spending Level'] = school_spending_bins

scores_by_school_spending = copy_school_sum.groupby(['Spending Level'])['Average Reading Score', 
                                                                       'Average Math Score',
                                                                       '% Passing Reading',
                                                                       '% Passing Math',
                                                                       '% Overall Passing Rate'].mean()
scores_by_school_spending

,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Overall Passing Rate
Spending Level,,,,,
$585-615,83.885211,83.599686,95.900287,94.230858,95.065572
$615-645,81.891436,79.079225,86.106569,75.668212,80.887391
$645-675,81.027843,76.997210,81.133951,66.164813,73.649382
<$585,83.933814,83.455399,96.610877,93.460096,95.035486


## Scores by School Size

* Perform the same operations as above, based on school size.

In [49]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [51]:
school_size_bins = pd.cut(copy_school_sum['Size'], size_bins, labels=group_names)

school_size_bins = pd.DataFrame(school_size_bins)

copy_school_sum['Size Level'] = school_size_bins

scores_by_school_size = copy_school_sum.groupby(['Size Level'])['Average Reading Score', 
                                                                       'Average Math Score',
                                                                       '% Passing Reading',
                                                                       '% Passing Math',
                                                                       '% Overall Passing Rate'].mean()
scores_by_school_size.head()

,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Overall Passing Rate
Size Level,,,,,
Large (2000-5000),81.344493,77.746417,82.766634,69.963361,76.364998
Medium (1000-2000),83.864438,83.374684,96.790680,93.599695,95.195187
Small (<1000),83.929843,83.821598,96.099437,93.550225,94.824831


## Scores by School Type

* Perform the same operations as above, based on school type.

In [58]:
#type_bins = [0, 1000, 2000]
#group_names = [Charter, District]

In [60]:
scores_by_school_type = copy_school_sum.copy()

scores_by_school_type = pd.DataFrame(scores_by_school_type)

scores_by_school_type = copy_school_sum.groupby(['School Type'])['Average Reading Score',
                                                         'Average Math Score',
                                                         '% Passing Reading', 
                                                         '% Passing Math',
                                                         '% Overall Passing Rate'].mean()
scores_by_school_type.head()

,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Overall Passing Rate
School Type,,,,,
Charter,83.896421,83.473852,96.586489,93.620830,95.103660
District,80.966636,76.956733,80.799062,66.548453,73.673757
